In [ ]:
# Use Python 3.12 environment
%pip install pillow mutagen librosa soundfile numpy

In [1]:
import shutil
import subprocess
import json
from io import BytesIO
from pathlib import Path
from PIL import Image
import librosa
import numpy as np

# -------------------------------------------------------------
# CONFIGURACIÓN DE DIRECTORIOS
# -------------------------------------------------------------
script_dir = Path().cwd()

input_dir = script_dir / "download_temp"
thumb_dir = script_dir / "download_temp"  # yt-dlp guarda aquí las thumbnails
output_dir = script_dir.parent / "Assets" / "StreamingAssets" / "Music"

input_dir.mkdir(exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)


In [2]:
# -------------------------------------------------------------
# 1. DESCARGAR AUDIO + MINIATURA ORIGINAL
# -------------------------------------------------------------
PLAYLIST_URL = "https://music.youtube.com/playlist?list=PL2SZb_BNUE8gqEwA_-QD8ym6BtURsRH2W&si=7JOgVgEkyHsvHAzP"

yt_dlp_command = [
    "yt-dlp",
    PLAYLIST_URL,
    "--format", "bestaudio",
    "--extract-audio",
    "--audio-format", "vorbis",
    "--postprocessor-args", "ffmpeg:-qscale:a 8",
    "--embed-metadata",
    "--add-metadata",
    "--write-thumbnail",
    "--no-overwrites",

    "-o", f"{input_dir}/%(title)s [%(id)s].%(ext)s"
]

print("\nDescargando playlist...\n")
subprocess.run(yt_dlp_command, check=True)
print("Descarga completada.\n")


Descargando playlist...

Descarga completada.



In [ ]:

# -------------------------------------------------------------
# TOOLS
# -------------------------------------------------------------
def make_square(img):
    w, h = img.size
    if w == h:
        return img
    side = min(w, h)
    left = (w - side) // 2
    top = (h - side) // 2
    return img.crop((left, top, left + side, top + side))


def calculate_bpm(path):
    try:
        y, sr = librosa.load(path, sr=None, mono=True)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        tempo_val = float(tempo[0]) if hasattr(tempo, "__len__") else float(tempo)
        return int(round(tempo_val))
    except Exception as e:
        print("Error BPM:", e)
        return None


def detect_key(y, sr):
    try:
        chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
        chroma_avg = chroma.mean(axis=1)
        note_names = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]
        idx = int(np.argmax(chroma_avg))
        return note_names[idx]
    except:
        return None


camelot_map = {
    "C":  "8B", "C#": "3B", "D":  "10B", "D#": "5B", "E":  "12B", "F":  "7B",
    "F#": "2B", "G":  "9B",  "G#": "4B", "A":  "11B","A#": "6B", "B":  "1B"
}


def compute_replaygain(path):
    try:
        y, sr = librosa.load(path, sr=None, mono=True)
        y = y / np.max(np.abs(y))
        loudness = np.mean(20*np.log10(np.abs(y) + 1e-6))
        track_gain = -18 - loudness
        peak = float(np.max(np.abs(y)))
        return f"{track_gain:.2f} dB", f"{peak:.6f}"
    except Exception as e:
        print("ReplayGain error:", e)
        return None, None


# -------------------------------------------------------------
# PROCESAR CADA TRACK
# -------------------------------------------------------------
def process_track(audio_path: Path):
    stem = audio_path.stem  # "Song Name [ID]"

    if "[" in stem and "]" in stem:
        yt_id = stem.split("[")[-1].replace("]", "")
    else:
        yt_id = ""

    thumb_path = input_dir / f"{stem}.webp"
    if not thumb_path.exists():
        thumb_path = None

    dst_audio = output_dir / audio_path.name
    shutil.copy2(audio_path, dst_audio)

    # ------------------------------------------
    # EXTRAER METADATOS OGG (artist, album)
    # ------------------------------------------
    from mutagen.oggvorbis import OggVorbis
    tags = OggVorbis(audio_path)

    artist = None
    album = None

    if "artist" in tags:
        artist = tags["artist"][0]
    elif "ARTIST" in tags:
        artist = tags["ARTIST"][0]

    if "album" in tags:
        album = tags["album"][0]
    elif "ALBUM" in tags:
        album = tags["ALBUM"][0]

    # ------------------------------------------
    # ANALIZAR AUDIO
    # ------------------------------------------
    bpm = calculate_bpm(audio_path)
    y, sr = librosa.load(audio_path, sr=None, mono=True)

    key = detect_key(y, sr)
    camelot = camelot_map.get(key, None) if key else None

    gain, peak = compute_replaygain(audio_path)

    # ------------------------------------------
    # PORTADA -> PNG
    # ------------------------------------------
    cover_filename = None

    if thumb_path:
        img = Image.open(thumb_path).convert("RGB")
        img_sq = make_square(img)
        cover_filename = f"{stem}.png"
        img_sq.save(output_dir / cover_filename, format="PNG")

    # ------------------------------------------
    # METADATA JSON
    # ------------------------------------------
    json_data = {
        "title": stem.rsplit(" [", 1)[0],
        "id": yt_id,
        "filename": audio_path.name,

        "artist": artist,
        "album": album,

        "bpm": bpm,
        "initial_key": key,
        "camelot": camelot,
        "replaygain_track_gain": gain,
        "replaygain_track_peak": peak,
        "cover": cover_filename
    }

    json_path = output_dir / f"{stem}.json"
    with open(json_path, "w", encoding="utf-8") as fp:
        json.dump(json_data, fp, indent=4, ensure_ascii=False)


# -------------------------------------------------------------
# LIMPIEZA PREVIA DE DESTINO
# -------------------------------------------------------------
for f in output_dir.iterdir():
    if f.is_file():
        f.unlink()


# -------------------------------------------------------------
# PROCESAR TODO LO DESCARGADO
# -------------------------------------------------------------
for f in input_dir.iterdir():
    if f.suffix.lower() == ".ogg":
        print("Procesando:", f.name)
        process_track(f)

# Limpieza
for f in input_dir.iterdir():
    if f.is_file():
        f.unlink()

print("\n✔ Procesamiento finalizado.")
print("Archivos listos en:", output_dir)


Procesando: All Falls Down [uHcP8XX5IJA].ogg
Procesando: Animal Rights [mczsm1sXbKQ].ogg
Procesando: ave That [HFRMw7UbEus].ogg
Procesando: Baddadan (Extended Mix) [wJSNHbiXTFI].ogg
Procesando: Big Beat Souffle [4OpwRRk6nvI].ogg
Procesando: Blame [70BASPcfWVQ].ogg
Procesando: Bloody Mary [1e32cgJrbBo].ogg
Procesando: Boom Boom Pow [QFi_LUksNX4].ogg
Procesando: Control Movement [qsVaIS9njHg].ogg
Procesando: Crescendolls [AuTRLEWSfTo].ogg
Procesando: D.O.D feat. Carla Monroe - Still Sleepless (Ekko & Sidetrack Remix) [qFPBWJNxL3s].ogg
Procesando: Disturbia [wmGjajXDHKo].ogg
Procesando: Don't Stop The Music [C75J03JXz24].ogg
Procesando: Don't Stop the Party [ZmIV4WwZYIM].ogg
Procesando: Don’t Stop The Party [YywgaoOoCbQ].ogg
Procesando: Driver [11CEsPNK7tI].ogg
Procesando: DVNO (Surkin Remix) [yyXbHm9FpcY].ogg
Procesando: Feeling for You [e_kIXeB9jeg].ogg
Procesando: Flashing Lights [cxKs2b5lRsA].ogg
Procesando: Ghetto Obsession 2006 [LjkG9v4Ec78].ogg
Procesando: Good Life [GTmbyuTKT_c].o